In [1]:
print("OK")

OK


In [2]:
%pwd

'd:\\NeuraVault\u200d🔥🧠\\MedBot\\research'

In [3]:
import os 
os.chdir("../")

In [5]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [6]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        data , 
        glob = "*.pdf",
        loader_cls= PyPDFLoader
    )
    documents = loader.load()
    return documents

In [7]:
extracted_text = load_pdf_file(data = 'Data/')

In [8]:

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap = 20
        )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks =text_split(extracted_text)
print("Length of text chunks : ",len(text_chunks))


Length of text chunks :  5860


In [17]:
# text_chunks 
from langchain.embeddings import HuggingFaceEmbeddings


In [16]:
def download_hugface_embedding():
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [19]:
embeddings = download_hugface_embedding()

d:\NeuraVault‍🔥🧠\MedBot\myenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\NeuraVault‍🔥🧠\MedBot\myenv\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rajve\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an a

In [20]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [22]:
# query_result

In [23]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [27]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
pc = Pinecone(api_key= PINECONE_API_KEY)
index_name = "medbot"
pc.create_index(
    name = index_name,
    dimension=384,
    metric = "cosine",
    spec = ServerlessSpec(
        cloud="aws",
        region = "us-east-1",
    )
)

In [28]:
# to store the embeddings into database
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding=embeddings
)

In [29]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding=embeddings
)

In [30]:
docsearch

In [31]:
retriever = docsearch.as_retriever(search_type= "similarity" , search_kwargs = {"k": 3})


In [33]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(metadata={'page': 39.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(metadata={'page': 38.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(metadata={'page': 37.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term fo

In [35]:
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

In [38]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature= 0.6,
    model = "llama3-70b-8192",
    api_key= "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"
)

In [41]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


In [42]:
system_prompt = (
    "You are an assistant for question answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the"
    "answer concise"
    "\n\n"
    "{context}"
    )

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [43]:
question_answer_chain = create_stuff_documents_chain(llm , prompt)
rag_chain = create_retrieval_chain(retriever , question_answer_chain)

In [45]:
response = rag_chain.invoke({"input": "What is acne?"})
print(response["answer"])

Acne is a common skin disease characterized by pimples on the face, chest, and back, occurring when the pores of the skin become clogged with oil, dead skin cells, and bacteria.


In [47]:
response = rag_chain.invoke({"input": "What is gigantism?"})
print(response["answer"])

Gigantism is a variant of acromegaly that occurs in children whose bony growth plates have not closed, resulting in exceptional growth of long bones.
